# **Project 3: Building a LangChain Function/Tool Calling Calculator with Google Gemini Flash**

## **Installing dependencies**

In [3]:
!pip install -U -q "google-generativeai>=0.7.2"

In [4]:
!pip install python-dotenv -q
!pip install langchain-community -U -q
!pip install google-generativeai -U -q
!pip install langchain -U -q
!pip install langchain_google_genai -U -q

In [5]:
!pip install langchain --upgrade -q

## **Securely Store and Retrieve Google API Key in Colab**

In [7]:
import os
from google.colab import userdata

google_api_key = userdata.get('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = google_api_key
# Retrieve the API key from the environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')


## **Define the Calculator Tool**

In [8]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval with restricted access to only arithmetic operations
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

## **Create the LangChain Tool Wrapper**

In [9]:
from langchain.tools import tool

# Define the tool using a decorator
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations and return results in a user-friendly format.
    Input: A mathematical expression as a string (e.g., "15 / 3").
    Output: A string with a concise result.
    """
    # Create an instance of the Calculator class
    calc = Calculator()
    # Call the calculate method to get the result
    return calc.calculate(expression)

## **Initialize the Google Gemini Flash Model**

In [10]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

# Initialize the Google Gemini Flash model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GOOGLE_API_KEY,
    temperature=0.7
)

# Confirm the model is initialized
print("Google Gemini Flash model initialized.")

Google Gemini Flash model initialized.


## **Set Up the ToolAgent**

In [12]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Define tools
tools = [calculator]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="openai-function-calling",
)

# Example usage: running a query with the agent
response = agent.invoke("What is 12 * 15?")
print("Response:", response)


Response: {'input': 'What is 12 * 15?', 'output': '180'}


## **Set Up Memory and Build the Conversational Chain**

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Set up memory to maintain conversation context
memory = ConversationBufferMemory()

# Build the conversational chain with the LLM (Gemini model), agent executor, and memory
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Example usage
response = chain.invoke("What is 12 * 15?")
print("Response:", response)

<ipython-input-13-66bc8c99c66a>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-13-66bc8c99c66a>:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 12 * 15?
AI:

> Finished chain.
Response: {'input': 'What is 12 * 15?', 'history': '', 'response': "12 * 15 is 180.  I arrived at that answer by performing the multiplication operation.  Specifically, I used an algorithm commonly taught in elementary school, which involves breaking down the multiplication into smaller, more manageable parts.  For example, I could have done (10 * 15) + (2 * 15) = 150 + 30 = 180.  My internal processes involved representing the numbers digitally and then applying the multiplication function.  While this might seem simple to you, it's a complex series of operations for me!"}


## **Test the Calculator Tool**

In [14]:
query = "What is 15 divided by 3?"

# Get response from the system
response = chain.invoke(query)

print("Query:", query)
print("Response:", response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at that answer by performing the multiplication operation.  Specifically, I used an algorithm commonly taught in elementary school, which involves breaking down the multiplication into smaller, more manageable parts.  For example, I could have done (10 * 15) + (2 * 15) = 150 + 30 = 180.  My internal processes involved representing the numbers digitally and then applying the multiplication function.  While this might seem simple to you, it's a complex series of operations for me!
Human: What is 15 divided by 3?
AI:

> Finished chain.
Query: What is 15 divided by 3?
Response: {'input': 'What is 15 divided by 3?', 'hi

## **Example with Multiple Queries**

In [39]:
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.run(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 15 divided by 3?
AI: 15 divided by 3 is 5.  That's a simple division problem;  15 contains three groups of 5.  You could also think of it as subtracting 3 from 15 repeatedly until you reach zero – you'd do it five times.  Basic arithmetic!
Human: What is 25 multiplied by 4?
AI: 25 multiplied by 4 is 100.  Think of it like this: 25 is a quarter of 100, and four quarters make a whole, so four 25s equal 100.  Or, you could use the standard multiplication method, multiplying 25 by 4: 4 times 5 is 20, carry the 2; 4 times 2 is 8, plus the carried 2 is 10.  So the answer is 100.  It's a fairly straightforward multiplication p

### **The End...**